In [1]:
import pyodbc
import pandas as pd
import numpy as np
from scipy import optimize
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
class DBConnector:
    def __init__(self, server_name, username, password, db_name, connection_str):
        self.sever_name = server_name
        self.username = username
        self.password = password
        self.db_name = db_name
        self.connection_str = connection_str
        self.connection = pyodbc.connect(self.connection_str)

    def test_connection(self):
        try:
            db = self.connection
            cursor = db.cursor()
            cursor.execute("SELECT @@VERSION")
            results = cursor.fetchone()
            # Check if anything at all is returned
            if results:
                return True
            else:
                return False
        except pyodbc.Error as ex:
            print(ex)
            print("Error in connection!")
            sqlstate = ex.args[0]
            if sqlstate == '28000':
                print("LDAP Connection failed: check password")
        return False

    def query(self, query_str):
        """This function returns a dataframe of result list"""
        return pd.read_sql_query(query_str, self.connection)

    def all_columns_name(self, table_name):
        """This function returns a dataframe of result list"""
        query_str = f"select COLUMN_NAME from INFORMATION_SCHEMA.COLUMNS where TABLE_NAME='{table_name}'"
        print("All columns names: ", query_str)
        return pd.read_sql_query(query_str, self.connection)

driver="SQL Server"
servername = 'QUOC-CUONG'
username = 'sa'
password = 'cuong300599'
db_name = 'OnlinePhoneShop'

str_for_connection = "Driver={driver};Server={servername};UID={username};PWD={password};Database={db_name};"\
    .format(driver=driver, servername=servername, username=username, password=password, db_name=db_name)

connector = DBConnector(servername, username, password, db_name, str_for_connection)

In [3]:
query_str = "select distinct u.user_id, p.product_id, dbo.fn_get_rating(u.user_id, p.product_id) as score from users u, products p"
df_users_products_ratings = connector.query(query_str)
df_users_products_ratings_pivotted = df_users_products_ratings.pivot(index='product_id', columns='user_id', values='score').fillna(0)
print("Number of products (rows): ", df_users_products_ratings_pivotted.shape[0])
print("Number of  users (columns): ", df_users_products_ratings_pivotted.shape[1])

# Save to csv file
df_users_products_ratings_pivotted.to_csv(r'D:\PhoneShopML\data\rating_matrix.csv', header=True, index=True, encoding='utf-8-sig')

df_users_products_ratings_pivotted.head(20)

Number of products (rows):  63
Number of  users (columns):  77


user_id         US041020210001  US061020210001  US071020210001  US191020210001  US191020210002  US191020210003  US191020210004  US191020210005  US191020210006  US191020210007  US191020210008  US191020210009  US191020210010  US191020210011  US191020210012  US251020210001  US251020210013  US251020210014  US251020210015  US251020210016  US251020210017  US251020210018  US251020210019  US251020210020  US271020210001  US271020210021  US271020210022  US271020210023  US271020210024  US271020210025  US271020210026  US271020210027  US271020210028  US271020210029  US271020210030  US271020210031  US271020210032  US271020210033  US271020210034  US271020210035  US271020210036  US271020210037  US281020210001  US281020210038  US281020210039  US281020210040  US281020210041  US281020210042  US281020210043  US281020210044  US281020210045  US281020210046  US281020210047  US281020210048  US281020210049  US281020210050  US281020210051  US281020210052  US281020210053  US281020210054  US281020210055  \
product_id                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
PD041020210001             4.0             0.0             2.5             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             1.0             0.0             0.0             0.0             0.0             1.5             0.0             0.0             0.0             0.0             0.0             1.5             0.0             0.0             0.0             0.0             2.0             0.0             0.0   
PD041020210002             2.5             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             2.5             0.0             0.0             2.5             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             4.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             3.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             1.0             0.0             0.0             0.0             0.0             0.0             1.0             0.0             2.5             0.0             0.0             0.0             2.0             2.5   
PD041020210003          

In [4]:
df_is_rated = df_users_products_ratings_pivotted.copy()
# df_is_rated.head(20)

column_names = df_users_products_ratings_pivotted.columns.values
print(column_names)

for name in column_names:
    df_is_rated.loc[df_is_rated[name] > 0, name] = 1

df_is_rated.head(20)

['US041020210001' 'US061020210001' 'US071020210001' 'US191020210001'
 'US191020210002' 'US191020210003' 'US191020210004' 'US191020210005'
 'US191020210006' 'US191020210007' 'US191020210008' 'US191020210009'
 'US191020210010' 'US191020210011' 'US191020210012' 'US251020210001'
 'US251020210013' 'US251020210014' 'US251020210015' 'US251020210016'
 'US251020210017' 'US251020210018' 'US251020210019' 'US251020210020'
 'US271020210001' 'US271020210021' 'US271020210022' 'US271020210023'
 'US271020210024' 'US271020210025' 'US271020210026' 'US271020210027'
 'US271020210028' 'US271020210029' 'US271020210030' 'US271020210031'
 'US271020210032' 'US271020210033' 'US271020210034' 'US271020210035'
 'US271020210036' 'US271020210037' 'US281020210001' 'US281020210038'
 'US281020210039' 'US281020210040' 'US281020210041' 'US281020210042'
 'US281020210043' 'US281020210044' 'US281020210045' 'US281020210046'
 'US281020210047' 'US281020210048' 'US281020210049' 'US281020210050'
 'US281020210051' 'US281020210052'

user_id         US041020210001  US061020210001  US071020210001  US191020210001  US191020210002  US191020210003  US191020210004  US191020210005  US191020210006  US191020210007  US191020210008  US191020210009  US191020210010  US191020210011  US191020210012  US251020210001  US251020210013  US251020210014  US251020210015  US251020210016  US251020210017  US251020210018  US251020210019  US251020210020  US271020210001  US271020210021  US271020210022  US271020210023  US271020210024  US271020210025  US271020210026  US271020210027  US271020210028  US271020210029  US271020210030  US271020210031  US271020210032  US271020210033  US271020210034  US271020210035  US271020210036  US271020210037  US281020210001  US281020210038  US281020210039  US281020210040  US281020210041  US281020210042  US281020210043  US281020210044  US281020210045  US281020210046  US281020210047  US281020210048  US281020210049  US281020210050  US281020210051  US281020210052  US281020210053  US281020210054  US281020210055  \
product_id                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
PD041020210001             1.0             0.0             1.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             1.0             0.0             0.0             0.0             0.0             1.0             0.0             0.0             0.0             0.0             0.0             1.0             0.0             0.0             0.0             0.0             1.0             0.0             0.0   
PD041020210002             1.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             1.0             0.0             0.0             1.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             1.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             1.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             0.0             1.0             0.0             0.0             0.0             0.0             0.0             1.0             0.0             1.0             0.0             0.0             0.0             1.0             1.0   
PD041020210003          

In [5]:
def cal_cost_func(params, *args):
    Y, R, num_users, num_movies, num_features, lmbda = args
    # Unfold the U and W matrices from params
    X = params[0:num_movies*num_features].reshape(num_movies, num_features, \
               order = 'F')
    Theta = params[num_movies*num_features:].reshape(num_users, num_features, \
                  order = 'F')
    
    squared_error = np.power(np.dot(X,Theta.T) - Y,2)
    # for cost function, sum only i,j for which R(i,j)=1
    J = (1/2.) * np.sum(squared_error * R)
    J = J + (lmbda/2.)*( np.sum( np.power(Theta, 2) ) + np.sum( np.power(X, 2) ) )
    
    return J

In [6]:
def cal_gradient_decent(params, *args):
    Y, R, num_users, num_movies, num_features, lmbda = args
    # Unfold the U and W matrices from params
    X = params[0:num_movies*num_features].reshape(num_movies, num_features, \
               order = 'F')
    Theta = params[num_movies*num_features:].reshape(num_users, num_features, \
                  order = 'F')
    
    X_grad_reg = lmbda * X
    Theta_grad_reg = lmbda * Theta
    
    X_grad = np.dot(( np.dot(X, Theta.T) - Y ) * R, Theta) + X_grad_reg
    Theta_grad =  np.dot((( np.dot(X, Theta.T) - Y ) * R).T, X) + Theta_grad_reg
    
    grad = np.hstack([X_grad.flatten('F'), Theta_grad.flatten('F')])
    
    return grad

In [7]:
def normalize_rating(Y, R):
    m, n = np.shape(Y)
    Ymean = np.zeros((m, 1))
    Ynorm = np.zeros((m, n))
    for i in range(m):
        idx = np.nonzero(R[i, :] == 1.0)[0]
        Ymean[i] = Y[i, idx].mean(axis = 0)
        Ynorm[i, idx] = Y[i, idx] - Ymean[i]
    return Ymean, Ynorm

In [8]:
Y = df_users_products_ratings_pivotted.to_numpy()
R = df_is_rated.to_numpy()

print("Rating numpy array: \n", Y)
print("Rating numpy array's shape: ", Y.shape)
print("Check rated numpy array\n", R)
print("Check rated numpy array's shape: \n", R.shape)

Rating numpy array: 
 [[4.  0.  2.5 ... 5.  5.  0. ]
 [2.5 0.  0.  ... 0.  0.  0. ]
 [4.  0.  0.  ... 5.  0.  5. ]
 ...
 [0.  0.  0.  ... 0.  5.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]]
Rating numpy array's shape:  (63, 77)
Check rated numpy array
 [[1. 0. 1. ... 1. 1. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 1. 0. 1.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Check rated numpy array's shape: 
 (63, 77)


In [9]:
Ymean, Ynorm = normalize_rating(Y, R)
num_products, num_users = np.shape(Y)
num_features = 3

X = np.random.standard_normal(num_products * num_features) 
Theta = np.random.standard_normal(num_users * num_features)

initial_parameters = np.hstack([X, Theta])
lmbda = 3
args = (Ynorm, R, num_users, num_products, num_features, lmbda)

theta = optimize.fmin_cg(cal_cost_func, initial_parameters, \
                         fprime=cal_gradient_decent, args=args, maxiter=100)

# Unfold the returned theta back into X and Theta
X = theta[0:num_products*num_features].reshape(num_products, num_features, \
           order = 'F')
Theta = theta[num_products*num_features:].reshape(num_users, num_features, \
              order = 'F')

Optimization terminated successfully.
         Current function value: 148.162436
         Iterations: 78
         Function evaluations: 126
         Gradient evaluations: 126


C:\Users\QUOCCU~1\AppData\Local\Temp/ipykernel_5680/663095750.py:7: RuntimeWarning: Mean of empty slice.
  Ymean[i] = Y[i, idx].mean(axis = 0)
c:\users\quoc cuong\appdata\local\programs\python\python39\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [10]:
# Make prediction
p = np.dot(X, Theta.T)
print("Shape of prediction array: ", p.shape)
print("Prediction Array: \n", p)

Shape of prediction array:  (63, 77)
Prediction Array: 
 [[ 0.09897835  0.17649368 -0.0898435  ...  1.36320415  1.43439496
   1.17710994]
 [ 0.0829273   0.17586319 -0.0887986  ...  1.33814317  1.42535759
   1.11816384]
 [ 0.11974363  0.12824181 -0.07311738 ...  1.12909864  1.14501841
   1.0602293 ]
 ...
 [ 0.06621871  0.13744393 -0.06798817 ...  1.02741549  1.09050267
   0.86879458]
 [ 0.07031022  0.13385092 -0.06803967 ...  1.02940213  1.08862224
   0.87695996]
 [ 0.00906628  0.02098765 -0.00993228 ...  0.15004911  0.15978399
   0.12630038]]


In [11]:
def get_product_info(product_id):
    query_str = f"select product_name, unit_price, label from products p where product_id='{product_id}'"
    df_info= connector.query(query_str)
    product_name, unit_price, label = df_info.values.tolist()[0]

    return product_name, unit_price, label

product_name, unit_price, label = get_product_info('PD271020210019')
print("Name: ", product_name)
print("Price: ", unit_price)
print("Label: ", label)

Name:  Điện thoại OPPO Reno6 Pro 5G
Price:  18490000.0
Label:  1


In [12]:
label2type={0:"Phụ kiện", 1: "Gaming/ Cấu hình cao", 2:"Giải trí thông thường", 3:"Chỉ nghe và gọi"}
# US281020210062
user_id = 'US281020210062'

In [13]:
print("User's ratings in the past:\n")
query_str = f"select user_id, product_id, unit_price, label, score from dbo.users_ratings where user_id='{user_id}'"
df_past_ratings = connector.query(query_str)
df_past_ratings["dedicated"] = df_past_ratings["label"].map(label2type)

df_past_ratings.head(20)

User's ratings in the past:



user_id      product_id  unit_price  label  score              dedicated
0  US281020210062  PD271020210041   4990000.0      2    4.5  Giải trí thông thường
1  US281020210062  PD161020210008    203000.0      0    4.5               Phụ kiện
2  US281020210062  PD151020210001  30990000.0      1    4.5   Gaming/ Cấu hình cao
3  US281020210062  PD081020210007   3790000.0      1    4.5   Gaming/ Cấu hình cao
4  US281020210062  PD041020210002   6790000.0      0    4.0               Phụ kiện

In [14]:
# Get the prediction of rating for the user
# order start from 0
import math

user_index_in_matrix = df_users_products_ratings_pivotted.columns.get_loc(user_id)
print("Index of user in matrix: ", user_index_in_matrix)

print("Y mean flatten: ", Ymean.flatten().shape)
print("P shape: ", p[:,user_index_in_matrix].shape)
user_prediction = p[:,user_index_in_matrix] + Ymean.flatten()
user_prediction = [x for x in user_prediction if str(x) != 'nan']

# Sort user prediction with descending order of score
sorted_indices = np.argsort(user_prediction)[::-1]
list_products = df_users_products_ratings_pivotted.index.values.tolist() 
n_top = 10
print("Collaborative Filtering recommendation products:\n")
cols = ['index', 'product_id', 'product_name', 'unit_price', 'dedicated', 'predicted_score']
recommended_products = []
for position in range(n_top):
    index = sorted_indices[position]
#     print(f"Product at Index: {index} - predict score: {user_prediction[index]} \nRecommend product: {get_product_name(list_products[index])}\n")
    product_name, unit_price, label = get_product_info(list_products[index])
    recommended_products.append([index, list_products[index], product_name, unit_price, label, user_prediction[index]])

df_recommended_products = pd.DataFrame(recommended_products, columns=cols)
df_recommended_products["dedicated"] = df_recommended_products["dedicated"].map(label2type)
df_recommended_products.head(n_top)

Index of user in matrix:  67
Y mean flatten:  (63,)
P shape:  (63,)
Collaborative Filtering recommendation products:



index      product_id                                       product_name  unit_price              dedicated  predicted_score
0     23  PD271020210017  Điện Thoại Oppo Reno 6 5G (8GB/128G) - Hàng Ch...  12990000.0   Gaming/ Cấu hình cao         5.000000
1     21  PD271020210015                     Điện thoại Xiaomi 11T 5G 256GB  11990000.0   Gaming/ Cấu hình cao         4.500000
2     25  PD271020210019                       Điện thoại OPPO Reno6 Pro 5G  18490000.0   Gaming/ Cấu hình cao         4.500000
3     22  PD271020210016  Điện Thoại Samsung A52S 5G 8GB/128GB - Hàng Ch...  10990000.0   Gaming/ Cấu hình cao         4.500000
4      7  PD111120210001  Điện Thoại OnePlus Nord CE 5G (12GB/256G) - Hà...   7990000.0   Gaming/ Cấu hình cao         4.450309
5     24  PD271020210018  Điện Thoại Xiaomi Mi 11 Lite 5G NE 6GB l 128GB...   9400000.0   Gaming/ Cấu hình cao         4.370873
6      9  PD151020210001  Điện Thoại iPhone 12 Pro Max 128GB - Hàng Chín...  30990000.0   Gaming/ Cấu hình cao         4.275501
7      4  PD041020210005                                           Vivo y12  10000000.0  Giải trí thông thường         4.230307
8      2  PD041020210003          Điện Thoại Vsmart Joy 4 - Hàng Chính Hãng   3059000.0  Giải trí thông thường         4.099419
9     31  PD271020210025  Điện thoại Vivo X70 Pro 5G (12GB/256GB) - Hàng...  17750000.0   Gaming/ Cấu hình cao         4.063895

# Recommendations for user has not had ratings

In [15]:
# Get list of users haven't rated before
query_str = "select user_id, username from users where user_id not in (select user_id from ratings)"
df_results = connector.query(query_str)

print("Users haven't rated any products: \n")
df_results.head(len(df_results.index))

Users haven't rated any products: 



user_id   username
0  US251020210019   hientran
1  US251020210017     hoangt
2  US251020210016  khangtran
3  US251020210013      mdang
4  US251020210020        myl
5  US281020210038       mytr
6  US251020210015   namthanh
7  US251020210018      ngoct
8  US251020210014    thienng
9  US191020210011       ynhi

In [16]:
# Get the prediction of rating for the user
# order start from 0
user_id = 'US251020210020'
user_index_in_matrix = df_users_products_ratings_pivotted.columns.get_loc(user_id)
print("Index of user in matrix: ", user_index_in_matrix)

print("Y mean flatten: ", Ymean.flatten().shape)
print("P shape: ", p[:,user_index_in_matrix].shape)
user_prediction = p[:,user_index_in_matrix] + Ymean.flatten()
user_prediction = [x for x in user_prediction if str(x) != 'nan']

# Sort user prediction with descending order of score
sorted_indices = np.argsort(user_prediction)[::-1]
list_products = df_users_products_ratings_pivotted.index.values.tolist() 
n_top = 10
print("Collaborative Filtering recommendation products:\n")
cols = ['index', 'product_id', 'product_name', 'unit_price', 'dedicated', 'predicted_score']
recommended_products = []
for position in range(n_top):
    index = sorted_indices[position]
#     print(f"Product at Index: {index} - predict score: {user_prediction[index]} \nRecommend product: {get_product_name(list_products[index])}\n")
    product_name, unit_price, label = get_product_info(list_products[index])
    recommended_products.append([index, list_products[index], product_name, unit_price, label, user_prediction[index]])

df_recommended_products = pd.DataFrame(recommended_products, columns=cols)
df_recommended_products["dedicated"] = df_recommended_products["dedicated"].map(label2type)
df_recommended_products.head(n_top)

Index of user in matrix:  23
Y mean flatten:  (63,)
P shape:  (63,)
Collaborative Filtering recommendation products:



index      product_id                                       product_name  unit_price             dedicated  predicted_score
0     23  PD271020210017  Điện Thoại Oppo Reno 6 5G (8GB/128G) - Hàng Ch...  12990000.0  Gaming/ Cấu hình cao         5.000000
1     21  PD271020210015                     Điện thoại Xiaomi 11T 5G 256GB  11990000.0  Gaming/ Cấu hình cao         4.500000
2     25  PD271020210019                       Điện thoại OPPO Reno6 Pro 5G  18490000.0  Gaming/ Cấu hình cao         4.500000
3     22  PD271020210016  Điện Thoại Samsung A52S 5G 8GB/128GB - Hàng Ch...  10990000.0  Gaming/ Cấu hình cao         4.500000
4     24  PD271020210018  Điện Thoại Xiaomi Mi 11 Lite 5G NE 6GB l 128GB...   9400000.0  Gaming/ Cấu hình cao         4.333333
5     31  PD271020210025  Điện thoại Vivo X70 Pro 5G (12GB/256GB) - Hàng...  17750000.0  Gaming/ Cấu hình cao         4.000000
6     16  PD191020210012  Ốp lưng dẻo trong suốt TPU loại tốt dành cho S...     12000.0              Phụ kiện         4.000000
7     27  PD271020210021   Điện Thoại iPhone 12 Pro 256GB - Hàng Chính Hãng  27500000.0  Gaming/ Cấu hình cao         4.000000
8     26  PD271020210020  Điện Thoại iPhone 13 Pro Max 1TB - Hàng Chính ...  47990000.0  Gaming/ Cấu hình cao         4.000000
9     30  PD271020210024  Điện Thoại Samsung Galaxy S21 Plus 5G (8GB/128...  20990000.0  Gaming/ Cấu hình cao         4.000000